### Exploring Airport data

#### Loading the Airport codes data

In [20]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 500)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T
from unidecode import unidecode
import re
import datetime as dt

In [2]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [4]:
df_apc =  spark.read.options(delimiter=",", header=True) \
                    .csv("Cleaned Data/I94_ports.csv")

In [5]:
df_apc.limit(5).toPandas()

,code,port,locality,state,country
0,CLG,None,Calgary,Alberta,Canada
1,EDA,None,Edmonton,Alberta,Canada
2,YHC,None,Hakai pass,British Columbia,Canada
3,HAL,None,Halifax,Nova Scotia,Canada
4,MON,None,Montreal,Quebec,Canada


#### Loading the Airport data

In [14]:
df_apd =  spark.read.options(delimiter="|", header=True, encoding="ISO-8859-1") \
                    .csv("Cleaned Data/Airports_Data/*.csv")

In [15]:
df_apd.limit(5).toPandas()

,ident,type,elevation_ft,continent,iso_country,iso_region,gps_code,iata_code,local_code,coordinates,nameL,municipalityL,nameE,municipalityE,country,state
0,00A,heliport,11,NA,US,US-PA,00A,None,00A,"-74.93360137939453, 40.07080078125",Total Rf Heliport,Bensalem,Total Rf Heliport,Bensalem,United States,Pennsylvania
1,00AA,small_airport,3435,NA,US,US-KS,00AA,None,00AA,"-101.473911, 38.704022",Aero B Ranch Airport,Leoti,Aero B Ranch Airport,Leoti,United States,Kansas
2,00AK,small_airport,450,NA,US,US-AK,00AK,None,00AK,"-151.695999146, 59.94919968",Lowell Field,Anchor Point,Lowell Field,Anchor Point,United States,Alaska
3,00AL,small_airport,820,NA,US,US-AL,00AL,None,00AL,"-86.77030181884766, 34.86479949951172",Epps Airpark,Harvest,Epps Airpark,Harvest,United States,Alabama
4,00AR,closed,237,NA,US,US-AR,None,None,None,"-91.254898, 35.6087",Newport Hospital & Clinic Heliport,Newport,Newport Hospital & Clinic Heliport,Newport,United States,Arkansas


In [16]:
df_apd.filter(F.col("iso_country")=="MX").limit(5).toPandas()

,ident,type,elevation_ft,continent,iso_country,iso_region,gps_code,iata_code,local_code,coordinates,nameL,municipalityL,nameE,municipalityE,country,state
0,AMC,small_airport,71,NA,MX,MX-SON,MMPE,PPE,83550,"-113.305177, 31.351987",Mar de Cortés International Airport,Puerto Peñasco,Mar de Cortes International Airport,Puerto Penasco,Mexico,Sonora
1,BHL,small_airport,34,NA,MX,MX-BCN,None,BHL,BAX,"-113.560997, 28.9786",Bahía de los Ángeles Airport,Bahía de los Ángeles,Bahia de los Angeles Airport,Bahia de los Angeles,Mexico,Baja California
2,BLM,small_airport,33,NA,MX,MX-BCN,None,None,BLM,"-113.528723717, 28.891952244799995",Bahia De Los Angelos South,None,Bahia De Los Angelos South,None,Mexico,Baja California
3,CYD,small_airport,575,NA,MX,MX-BCS,None,None,None,"-112.8851, 27.2906",San Ignacio Downtown Airstrip,Mulegé,San Ignacio Downtown Airstrip,Mulege,Mexico,Baja California Sur
4,LOM,small_airport,6227,NA,MX,MX-JAL,None,LOM,LMO,"-101.9441, 21.2581",Francisco Primo de Verdad y Ramos Airport,Lagos de Moreno,Francisco Primo de Verdad y Ramos Airport,Lagos de Moreno,Mexico,Jalisco


    Merging both the dataframes by the city. state, and country

In [25]:
cond = [df_apc.locality==df_apd.municipalityE, df_apc.state==df_apd.state, df_apc.country==df_apd.country]
df_merged = df_apd.join(F.broadcast(df_apc), cond, "inner")

In [26]:
df_merged.limit(5).toPandas()

,ident,type,elevation_ft,continent,iso_country,iso_region,gps_code,iata_code,local_code,coordinates,nameL,municipalityL,nameE,municipalityE,country,state,code,port,locality,state,country
0,CA-0005,closed,None,NA,CA,CA-AB,None,None,None,"-114.13300323486328, 51.0093994140625",RCAF Station Lincoln Park,Calgary,RCAF Station Lincoln Park,Calgary,Canada,Alberta,CLG,None,Calgary,Alberta,Canada
1,CA-0088,small_airport,210,NA,CA,CA-ON,None,None,None,"-75.29694366455078, 45.391666412353516",Navan Bearbrook Airport,Ottawa,Navan Bearbrook Airport,Ottawa,Canada,Ontario,OTT,None,Ottawa,Ontario,Canada
2,CA-0125,closed,2225,NA,CA,CA-AB,None,None,FR3,"-113.233002, 53.583302",Bremner Airport,Edmonton,Bremner Airport,Edmonton,Canada,Alberta,EDA,None,Edmonton,Alberta,Canada
3,CA-0126,closed,None,NA,CA,CA-AB,None,None,None,"-113.46700286865234, 53.68330001831055",Namao Airport,Edmonton,Namao Airport,Edmonton,Canada,Alberta,EDA,None,Edmonton,Alberta,Canada
4,CA-0396,closed,None,NA,CA,CA-BC,None,None,None,"-122.933339775, 49.2691976954",Burnaby (Terminal) Heliport,Vancouver,Burnaby (Terminal) Heliport,Vancouver,Canada,British Columbia,VCV,None,Vancouver,British Columbia,Canada


In [27]:
df_merged.count()

194

In [28]:
df_merged.columns

['ident',
 'type',
 'elevation_ft',
 'continent',
 'iso_country',
 'iso_region',
 'gps_code',
 'iata_code',
 'local_code',
 'coordinates',
 'nameL',
 'municipalityL',
 'nameE',
 'municipalityE',
 'country',
 'state',
 'code',
 'port',
 'locality',
 'state',
 'country']

In [30]:
df_merged.filter(F.col("iso_country")=="MX").limit(10).toPandas()

,ident,type,elevation_ft,continent,iso_country,iso_region,gps_code,iata_code,local_code,coordinates,nameL,municipalityL,nameE,municipalityE,country,state,code,port,locality,state,country
0,MMGL,large_airport,5016,NA,MX,MX-JAL,MMGL,GDL,None,"-103.31099700927734, 20.521799087524414",Don Miguel Hidalgo Y Costilla International Ai...,Guadalajara,Don Miguel Hidalgo Y Costilla International Ai...,Guadalajara,Mexico,Jalisco,GDL,Guadalajara International Airport,Guadalajara,Jalisco,Mexico
1,MMHO,large_airport,627,NA,MX,MX-SON,MMHO,HMO,None,"-111.047996521, 29.095899581900003",General Ignacio P. Garcia International Airport,Hermosillo,General Ignacio P. Garcia International Airport,Hermosillo,Mexico,Sonora,HMO,General Pesqueira Garcia Airport,Hermosillo,Sonora,Mexico
2,MMUN,large_airport,22,NA,MX,MX-ROO,MMUN,CUN,None,"-86.8770980835, 21.036500930800003",Cancún International Airport,Cancún,Cancun International Airport,Cancun,Mexico,Quintana Roo,CUN,None,Cancun,Quintana Roo,Mexico
3,MX-0158,small_airport,978,NA,MX,MX-SON,None,None,BRE,"-110.80762, 29.28228",Las Brechas Airport,Hermosillo,Las Brechas Airport,Hermosillo,Mexico,Sonora,HMO,General Pesqueira Garcia Airport,Hermosillo,Sonora,Mexico
4,MX-0296,closed,560,NA,MX,MX-SON,None,None,PAV,"-111.0527, 28.9979",Pavlovich Aviation Airport,Hermosillo,Pavlovich Aviation Airport,Hermosillo,Mexico,Sonora,HMO,General Pesqueira Garcia Airport,Hermosillo,Sonora,Mexico
5,MX-0511,heliport,5280,NA,MX,MX-JAL,None,None,HAZ,"-103.376303, 20.702397",Americas Country Heliport,Guadalajara,Americas Country Heliport,Guadalajara,Mexico,Jalisco,GDL,Guadalajara International Airport,Guadalajara,Jalisco,Mexico
6,MX-0639,heliport,5387,NA,MX,MX-JAL,None,None,HGH,"-103.399016, 20.680883",Hospital Angeles del Carmen Helipad,Guadalajara,Hospital Angeles del Carmen Helipad,Guadalajara,Mexico,Jalisco,GDL,Guadalajara International Airport,Guadalajara,Jalisco,Mexico
7,MX-0733,small_airport,233,NA,MX,MX-SON,None,None,CSA,"-110.534182, 28.089865",Campo Sonora Airstrip,Hermosillo,Campo Sonora Airstrip,Hermosillo,Mexico,Sonora,HMO,General Pesqueira Garcia Airport,Hermosillo,Sonora,Mexico
8,MX-0758,closed,5814,NA,MX,MX-JAL,None,None,HRX,"-103.394021, 20.665893",Hotel Riu Palace Heliport,Guadalajara,Hotel Riu Palace Heliport,Guadalajara,Mexico,Jalisco,GDL,Guadalajara International Airport,Guadalajara,Jalisco,Mexico
9,MX-0782,small_airport,295,NA,MX,MX-SON,None,None,AES,"-111.253725, 28.952572",El Socorro Airport,Hermosillo,El Socorro Airport,Hermosillo,Mexico,Sonora,HMO,General Pesqueira Garcia Airport,Hermosillo,Sonora,Mexico
